# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

When solving this task, we expect you'll face (and successfully deal with) some problems or make up the ideas of the model improvement. Some of them are:

- solving a problem of n-grams frequencies storing for a large corpus;
- taking into account keyboard layout and associated misspellings;
- efficiency improvement to make the solution faster;
- ...

Please don't forget to describe such cases, and what you decided to do with them, in the Justification section.

##### IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

# Improved approach (my solution)

In [17]:
import nltk
from collections import Counter
from nltk.util import ngrams
from difflib import get_close_matches
import re
from itertools import product
import string
import random
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [18]:
def preprocess(data):
  tokens = nltk.word_tokenize(data.lower())
  return [token for token in tokens if token.isalpha()]

In [19]:
qwerty_neighbors = {
    'a': "qwsz", 'b': "vghn", 'c': "xdfv", 'd': "ersfxc",
    'e': "rdsw", 'f': "rtgvcd", 'g': "tyhbfv", 'h': "yujnbg",
    'i': "uojk", 'j': "uikmnh", 'k': "iolmj", 'l': "opk",
    'm': "njk", 'n': "bhjm", 'o': "iplk", 'p': "ol",
    'q': "wa", 'r': "etdf", 's': "awedxz", 't': "rygf",
    'u': "yihj", 'v': "cfgb", 'w': "qase", 'x': "zsdc",
    'y': "tuhg", 'z': "asx"
}

# Function to get substitution weight based on keyboard proximity
def key_distance(c1, c2):
    """Return a weight factor based on keyboard proximity."""
    if c1 == c2:
        return 1.0  # No substitution
    elif c2 in qwerty_neighbors.get(c1, ''):
        return 1.2  # Close key, lower penalty
    else:
        return 0.8

In [20]:
# receives a pair of words and receives the corrected one
def correct_bigram(words):
  if len(words) != 2:
      print("two words expected")
      return
  cands = []
  for word in words:
      cands.append(candidates(word))
  i,j = cands
  #print(i,"-------",j)
  permutations = [(x, y) for x, y in product(i, j)]
  return max(permutations, key = bigramProbability)

def correct_single(word):
  cands = candidates(word)
  return max(cands, key = singleProbability)

In [21]:
# calculates the probability for each bigram, taking edit distance int account
def bigramProbability(words):
    if len(words) != 2:
      print("two words expected")
      return
    bigram = (words[0][0],words[1][0])
    return (bg[bigram]/words_count) * words[0][1] * words[1][1]
# calculates the probability for single word
def singleProbability(word):
    global word_freq,words_count
    return (words_freq[word[0]]/words_count) * word[1]

# return candidates for word correction
def candidates(word):
    res = edits1(word,pas = 1)
    return (known([(word,1)]) or known(res) or known(edits2(res)) or {(word,1)})

# check if the word is valid
def known(words):
    return set(w for w in words if w[0] in words_freq)

# 1 edit corrections
def edits1(word,pas): #called for each word separatly
    letters    = 'abcdefghijklmnopqrstuvwxyz'

    if pas == 1:
      splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
      deletes    = [(L + R[1:],0.8)               for L, R in splits if R] #deletes are fined by 0.8 since its less likely in my practice
      transposes = [(L + R[1] + R[0] + R[2:],1) for L, R in splits if len(R)>1] #transposes are left with weight 1, since they are pretty common, considered as base
      replaces   = [(L + c + R[1:],key_distance(R[0], c))           for L, R in splits if R for c in letters] # all replaces find by keyboard layout, from 0.8 to 1.2, so its the only one what may have positive impact on the probability, since if the latter is really close, it more likely to be just a typo
      inserts    = [(L + c + R,0.8)               for L, R in splits for c in letters] # all insertions fined by 0.8 similar to deletes
      if word == "rotted":
        print(replaces)
    elif pas == 2:                                                                  #second pass
      splits     = [(word[0][:i], word[0][i:])    for i in range(len(word[0]) + 1)]
      deletes    = [(L + R[1:],word[1]*0.8)               for L, R in splits if R]
      transposes = [(L + R[1] + R[0] + R[2:],word[1]*1) for L, R in splits if len(R)>1]
      replaces   = [(L + c + R[1:],word[1]*key_distance(R[0], c))           for L, R in splits if R for c in letters]
      inserts    = [(L + c + R,word[1]*0.8)           for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

# 2 edit corrections
def edits2(res):
    a = {e2 for e1 in res for e2 in edits1(e1,pas = 2)}
    return a

# Helper functions to process text and pass it to solution

In [22]:

# receives a text, divide it on bigrams and corrects the text
def fix_typos(text):
    # split text into words and punctuation while preserving spaces
    tokens = re.findall(r"\w+\'\w+|\w+|[.,!?;]", text.lower())

    fixed_tokens = []
    for i in range(len(tokens) - 1):
        word1, word2 = tokens[i], tokens[i + 1]

        # ensure both are words (not punctuation)
        if word1.isalpha() and word2.isalpha():
            corrected = correct_bigram([word1, word2])
            if corrected:
                fixed_tokens.append(corrected[0][0])  # first word from corrected bigram
                tokens[i + 1] = corrected[1][0]  # replace next word directly in tokens
                continue  # Skip appending word1 again
        elif word1.isalpha() and not word2.isalpha():
            corrected = correct_single(word1)
            if corrected:
                fixed_tokens.append(corrected[0])  # extract only the word
            else:
                fixed_tokens.append(word1)
        elif not word1.isalpha() and word2.isalpha():
            corrected = correct_single(word2)
            if corrected:
                tokens[i + 1] = corrected[0]  # extract only the word
            fixed_tokens.append(word1)
        else:
            fixed_tokens.append(word1)

    # Append the last token
    fixed_tokens.append(tokens[-1])
    fixed_text = " ".join(fixed_tokens)
    fixed_text = re.sub(r"\s+([{}])".format(re.escape(string.punctuation)), r"\1", fixed_text)
    return fixed_text

In [23]:
# compares two texts and prints mismatches
def compare_text(correct, text2, show_mismatches=False):
    def preprocess(text):
        """Tokenizes, lowercases, and removes punctuation from text."""
        return [word.lower().strip(string.punctuation) for word in text.split()]

    correct_words = preprocess(correct)
    text2_words = preprocess(text2)

    matches = 0
    mismatches = []

    for c, t in zip(correct_words, text2_words):
        if c == t:
            matches += 1
        else:
            mismatches.append((c, t,))

    total_words = max(len(correct_words), len(text2_words))
    accuracy = matches / total_words if total_words > 0 else 0

    print(f"Correct words: {matches}/{total_words}")
    print(f"Accuracy: {accuracy:.2%}")

    if show_mismatches and mismatches:
        print("\nMismatched words:")
        for correct_word, test_word in mismatches:
            print(f"Expected: '{correct_word}' → Found: '{test_word}', ")

# Norvig Solution

In [24]:
import re
from collections import Counter

def to_words(text): return re.findall(r'\w+', text.lower())

words_counter = Counter(to_words(open('big.txt').read()))

def P(word, N=sum(words_counter.values())):
    "Probability of `word`."
    return words_counter[word] / N

def correctionN(word):
    "Most probable spelling correction for word."
    return max(candidatesN(word), key=P)

def candidatesN(word):
    "Generate possible spelling corrections for word."
    return (knownN([word]) or knownN(edits1N(word)) or knownN(edits2N(word)) or [word])

def knownN(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in words_counter)

def edits1N(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2N(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1N(word) for e2 in edits1N(e1))

def Norvig_fix_typos(text):
    words_in_text = re.findall(r'\w+\'\w+|\w+', text.lower())
    corrected_words = [correctionN(word) for word in words_in_text]

    return ' '.join(corrected_words)

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

I used big.txt text to create a set of words and bigrams, the one from useful links. It gives me 371000 bigrams, not very big number, during testing i discovered that large number of test cases are not covered. I suppose even if dataset will be several times larger, the problem will remain. So, to test my improvements and compaer it to Norvig's solution, i will use text from big.txt with some number of intended typos.



In my solution i used bigram Counter along with weighted edit distance to get the probability for pair of words. With this slight modifications, bigram approach still process combiantions of just two words at a time, so the complexity remains manageable and doesn't escalate too much. At the same time, weighted edit distance will be more effective for natural typos, since it takes into account keyboard layoyt and some other minor things(different cost for replace,insertion etc). For edit2 i multiply probability of edit1 by probability of current typo type, so two typos in the word are less probable.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity (or just take another dataset). Compare your solution to the Norvig's corrector, and report the accuracies.

# Generating test sets

In [25]:
def introduce_typos(text, error_rate=0.1):
    words = re.findall(r'\w+|\s+|[.,!?;]', text)
    typo_text = []

    for word in words:
        if word.isalpha() and random.random() < error_rate:  # errors randomly
            typo_text.append(make_typo(word))
        else:
            typo_text.append(word)

    return "".join(typo_text)

def make_typo(word):
    typo_type = random.choice(["insert", "delete", "swap"])
    index = random.randint(0, len(word) - 1)

    if typo_type == "insert":
        letter = random.choice("abcdefghijklmnopqrstuvwxyz")
        return word[:index] + letter + word[index:]
    elif typo_type == "delete" and len(word) > 1:
        return word[:index] + word[index + 1:]
    elif typo_type == "swap" and len(word) > 1:
        if index == len(word) - 1:
            index -= 1
        return word[:index] + word[index + 1] + word[index] + word[index + 2:]
    return word

# read text and generate outputs
def generate_texts(file_path, size, error_rate=0.2):
    with open(file_path, "r", encoding="utf-8") as f:
        data = f.read()

    # remove sonething like "#--, \n"
    data = re.sub(r'[#-]+|\n+', ' ', data).strip()
    data = ' '.join(word for word in data.split() if word.isalpha())

    if len(data) <= size:
        selected_text = data
    else:
        start = random.randint(0, len(data) - size)
        selected_text = data[start:start + size]

    good_text = selected_text
    bad_text = introduce_typos(selected_text, error_rate)

    return good_text, bad_text


In [26]:
f = open("big.txt", "r")
data = f.read()
tokens = preprocess(data)
words_freq = Counter(tokens)
bg = Counter(ngrams(tokens,2))
words_count = len(tokens)

My handwritten test sample, i typed text by myslef very fast, so there is plenty **natural** typos

In [27]:
good_text = "Not to be outdone by the mariners of the deep, western souts searched for overland routes to the Pacific. Zebulon Pike, explorer and pathfinder, by his expedition into the Southwest during Jefferson's administration, had discovered the resources of New Spain and had shown his countrymen how easy it was to reach Santa Fe from the upper waters of the Arkansas River. Not long afterward, traders laid open the route, making Franklin, Missouri, and later Fort Leavenworth the starting point. Along the trail, once surveyed, poured caravans heavily guarded by armed men against marauding Indians. Sand storms often wiped out all signs of the route; hunger and thirst did many a band of wagoners to death; but the lure of the game and the profits at the end kept the business thriving. Huge stocks of cottons, glass, hardware, and ammunition were drawn almost across the continent to be exchanged at Santa Fe for furs, Indian blankets, silver, and mules; and many a fortune was made out of the traffic."
bad_text = "Not to be outdone bu tht marines of te dep, wastent scouts searched for overland routes to the pacifiv. Zebulon Pike, explorer and oathdinder, by his expedition into the SOuthwest during Jefferson's administration, had discovered the resources of New Spain and had SHown jis countrymen haw easy it was to reafch santa fe from the upepr waters fo the arkansas river. Not lonag afereward, traders laid opne the route, markin Franklin, Missousuri, and later fort Leavenworth the starinf pount. Along ht train, once survevyed, poursed caravans heavely guarded by armed men againt marauding Indians. Sand storms often wiped out all signs of th route; unger and thirt did many a band of wgoners to death; bat the lure of the game adn the progits at the end kept the business thrivign. hife stocks of cottonfs, galss, hardwate, und ammunitaion, wer drawn almsor across the contient to be exchahnged at Santa fe for firs, Indian balnkets, silver, and miles; and many a fortune was amde out of the traffic."

print("orogonal accuracy: ", end = "")
compare_text(good_text,bad_text)
print("\nsolution accuracy: ",end = "")
corrected_text = fix_typos(bad_text)
compare_text(good_text,corrected_text)
print("\nNorvig accuracy: ",end = "")
norvig_text = Norvig_fix_typos(bad_text)
compare_text(good_text,norvig_text)



orogonal accuracy: Correct words: 119/168
Accuracy: 70.83%

solution accuracy: Correct words: 149/168
Accuracy: 88.69%

Norvig accuracy: Correct words: 142/168
Accuracy: 84.52%


My improved approach predicted 31/49 words, which gives accuracy about 63%. Norvig solution gets 23/49, which give accuracy about 47%.
It can be seen that introducing the keyboard layout as a factor leads to better results when fixing human typos. Most of the rest unfixed typos are just some words with typo what still occur to be existing word. I tried to generate candidates even for correct words, but it ends up replacing most of the other correct words, resulting in worse accuracy.

accuracy on **random** tests

In [28]:
good_text, bad_text = generate_texts("big.txt",10000)

print("original accuracy: ", end = "")
compare_text(good_text,bad_text)
print("\nsolution accuracy: ",end = "")
corrected_text = fix_typos(bad_text)
compare_text(good_text,corrected_text)
print("\nNorvig accuracy: ",end = "")
norvig_text = Norvig_fix_typos(bad_text)
compare_text(good_text,norvig_text)

original accuracy: Correct words: 1590/1982
Accuracy: 80.22%

solution accuracy: Correct words: 1873/1982
Accuracy: 94.50%

Norvig accuracy: Correct words: 1846/1982
Accuracy: 93.14%


On random test the improvement is not so obvious, and its understandable, the typos here are randomly generated, not representing how humans are usually making them. I observed that in this case my approach either the same or just a tiny bit better.

#### Useful resources (also included in the archive in moodle):

1. [Possible dataset with N-grams](https://www.ngrams.info/download_coca.asp)
2. [Damerau–Levenshtein distance](https://en.wikipedia.org/wiki/Damerau–Levenshtein_distance#:~:text=Informally%2C%20the%20Damerau–Levenshtein%20distance,one%20word%20into%20the%20other.)